In [ ]:
import cv2
import numpy as np

import os

from google.colab import drive
drive.mount('/content/gdrive')

# Define the input and output directories
input_dir = '/content/gdrive/MyDrive/dataset'
output_dir = '/content/gdrive/MyDrive/out'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define the morphological kernel size
kernel_size = 10

# Process each image in the input directory
for filename in os.listdir(input_dir):
    # Load the input image
    img = cv2.imread(os.path.join(input_dir, filename))
    img =cv2.resize(img,(300,300))

    # Convert the color image to grayscale using the weighted process
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.addWeighted(gray, 0.3, gray, 0.59, 0)
    gray = cv2.addWeighted(gray, 1, gray, 0.11, 0)

    # Apply morphological black-hat transformation to identify hair contour
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)

    # Threshold the image to create a binary mask
    _, mask = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)


    # Use the Fast-Marching Method to inpaint the hair regions
    mask = cv2.inpaint(img, mask, 3, cv2.INPAINT_TELEA)

    # Save the result to disk
    cv2.imwrite(os.path.join(output_dir, filename), mask)



cv2.destroyAllWindows()

Mounted at /content/gdrive


In [ ]:
import cv2
import os

# Set the directory containing the input images
input_dir = '/content/gdrive/MyDrive/out'

# Set the directory to save the output images
output_dir = '/content/gdrive/MyDrive/blurremoved'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Set the kernel size for the Gaussian blur filter
kernel_size = (5,5)

# Set the parameters for the unsharp masking filter
unsharp_weight = 1.5
blur_weight = -0.5

# Loop over each image in the input directory
for filename in os.listdir(input_dir):
    # Load the image
    img = cv2.imread(os.path.join(input_dir, filename))

    # Apply a Gaussian blur filter
    blur = cv2.GaussianBlur(img, kernel_size, 0)

    # Apply an unsharp masking filter
    unsharp_mask = cv2.addWeighted(img, unsharp_weight, blur, blur_weight, 0)

    # Save the output image
    cv2.imwrite(os.path.join(output_dir, filename), unsharp_mask)

In [ ]:
import cv2
import os
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')


# Define input and output directories
input_dir = '/content/gdrive/MyDrive/blurremoved'
output_dir = '/content/gdrive/MyDrive/grabcut'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for filename in os.listdir(input_dir):
    # Read the image file
    image = cv2.imread(os.path.join(input_dir, filename))



    # Convert the image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Apply adaptive histogram equalization
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    hsv[:,:,2] = clahe.apply(hsv[:,:,2])

    # Convert the image back to BGR color space
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    # Apply Grab-cut segmentation
    mask = np.zeros(image.shape[:2],np.uint8)
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    rect = (10,12,250,270)
    cv2.grabCut(bgr,mask,rect,bgdModel,fgdModel,30,cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    segmented = bgr*mask2[:,:,np.newaxis]

    # Apply a threshold of 1 to the mask
    mask2[mask2 ==  1] = 255
    mask2[mask2 == 0] = 0

    # Save the segmented image to disk
    output_filename = os.path.splitext(filename)[0] + '_seg.png'
    cv2.imwrite(os.path.join(output_dir, output_filename),segmented)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:

import cv2
import os
import numpy as np
input_dire='/content/gdrive/MyDrive/dataset'
input_dir = '/content/gdrive/MyDrive/grabcut'
output_dir = '/content/gdrive/MyDrive/originalshape'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for filename in os.listdir(input_dire):
    # Load the input image
    img = cv2.imread(os.path.join(input_dire, filename))
    new_width=img.shape[0]
    new_height=img.shape[1]
    # Loop through each group of four images
    for filename2 in os.listdir(input_dir):
        filename1=filename2[0:12]+".JPG"
        if(filename==filename1):
            img1= cv2.imread(os.path.join(input_dir, filename2))
            out=cv2.resize(img1,(new_height,new_width))
            cv2.imwrite(os.path.join(output_dir, filename2), out)

In [ ]:
from PIL import Image
import os

# Define the directory containing the pre-processed skin images
input_dir = "/content/gdrive/MyDrive/originalshape"

# Define the output directory where augmented images will be saved
output_dir = "/content/gdrive/MyDrive/augmentation"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define the rotation angles
angles = [0, 90, 180, 270]

# Loop over all images in the input directory
for filename in os.listdir(input_dir):
    # Load the image
    image = Image.open(os.path.join(input_dir, filename))

    # Loop over the rotation angles
    for angle in angles:
        # Rotate the image
        rotated_image = image.rotate(angle)

        # Generate a new filename for the rotated image
        new_filename = filename[:-4] + "_" + str(angle) + ".jpg"

        # Save the rotated image to the output directory
        rotated_image.save(os.path.join(output_dir, new_filename))

In [ ]:
import cv2
import numpy as np
import glob

a=[]
b=[]
c=[]
d=[]
ABCD = []
#------------------------------------------------------------------------ASYMMETRY----------------------------------------------------
for fil in glob.glob('/content/drive/MyDrive/sample/*.JPG'):
    # Load the image
    img1 = cv2.imread(fil)
    img = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

    # Calculate the center of gravity
    M = cv2.moments(img)
    cg_x = int(M['m10']/M['m00'])
    cg_y = int(M['m01']/M['m00'])

    # Draw two orthogonal axes
    cv2.line(img, (cg_x - 100, cg_y), (cg_x + 100, cg_y), (0, 0, 255), 2)
    cv2.line(img, (cg_x, cg_y - 100), (cg_x, cg_y + 100), (0, 0, 255), 2)

    # Divide the lesion into four quadrants
    quadrants = [
        img[0:cg_y, 0:cg_x],
        img[0:cg_y, cg_x:],
        img[cg_y:, 0:cg_x],
        img[cg_y:, cg_x:]
    ]

    # Calculate the mean and standard deviation for each quadrant
    mean_bright = []
    mean_color = []
    std_bright = []
    std_color = []

    for quadrant in quadrants:
        if quadrant.ndim == 2:
            # Quadrant is grayscale, with only one channel
            mean_bright.append(np.mean(quadrant))
            mean_color.append(0)
            std_bright.append(np.std(quadrant))
            std_color.append(0)
        else:
            # Quadrant is color, with three channels
            mean_bright.append(np.mean(quadrant[:,:,0]))
            mean_color.append(np.mean(quadrant[:,:,1:], axis=(0,1)))
            std_bright.append(np.std(quadrant[:,:,0]))
            std_color.append(np.std(quadrant[:,:,1:], axis=(0,1)))

    # Calculate the asymmetry score for each axis
    axis1_score = abs(mean_bright[0]-mean_bright[2])/max(std_bright[0], std_bright[2])
    axis2_score = abs(mean_bright[1]-mean_bright[3])/max(std_bright[1], std_bright[3])
    if(axis1_score>0.2 and axis2_score >0.2):
      asymmetry_score=2;
    elif(axis1_score>0.2 or axis2_score>0.2):
      asymmetry_score=1;
    else:
      asymmetry_score=0;
    print("Asymmetry score:", asymmetry_score)
    a.append(asymmetry_score)



In [ ]:
#----------------------------------------BORDER-----------------------------------------------

for fil in glob.glob('/content/drive/MyDrive/output_directory3/*.png'):
  # Load the image
  img = cv2.imread(fil)

  # Divide the image into eight equal parts (slices)
  slices = []
  for i in range(8):
    y1 = i * img.shape[0] // 8
    y2 = (i+1) * img.shape[0] // 8
    slice = img[y1:y2, :]
    slices.append(slice)

  # Initialize the border score
  border_score = 0

  # Loop over each slice and calculate the border score
  for slice in slices:
    # Convert the slice to grayscale
    gray = cv2.cvtColor(slice, cv2.COLOR_BGR2GRAY)

    # Apply a Canny edge detector to the slice
    edges = cv2.Canny(gray, 100, 200)

    # Calculate the number of contours in the slice
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    num_contours = len(contours)

    # Calculate the border score for this slice
    slice_border_score = 1 if num_contours > 1 else 0

    # Add the slice border score to the overall border score
    border_score += slice_border_score

  # Print the overall border score
  print('Border score:', border_score)
  b.append(border_score)


In [ ]:
import cv2
import numpy as np
import glob

# Define the suspicious colors
suspicious_colors = {
    "white": (255, 255, 255),
    "red": (0, 0, 255),
    "black": (0, 0, 0),
    "light_brown": (165, 123, 63),
    "dark_brown": (60, 20, 20),
    "blue_gray": (100, 149, 237)
}

# Load all the images in the directory
for fil in glob.glob('/content/drive/MyDrive/sample/*.JPG'):
    # Load the image
    img = cv2.imread(fil)

    # Convert the image to HSV color space
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Initialize the color score to 0
    color_score = 0

    # Calculate the total number of pixels in the image
    num_pixels = img.shape[0] * img.shape[1]

    # Check for each suspicious color
    for color_name, color_value in suspicious_colors.items():
        # Create a mask for the color
        lower_range = np.array([color_value[0]-10, 100, 100])
        upper_range = np.array([color_value[0]+10, 255, 255])
        mask = cv2.inRange(hsv_img, lower_range, upper_range)

        # Calculate the number of pixels of the color
        num_color_pixels = cv2.countNonZero(mask)

        # Check if the color is present in the image
        if num_color_pixels >= num_pixels * 0.05:
            color_score += 1

    print("Color score:", color_score)
    c.append(color_score)

In [ ]:
#-------------------------------------------DERMOSCOPIC STRUCTURES---------------------------------------
for fil in glob.glob('/content/drive/MyDrive/output_directory3/*.png'):
    img = cv2.imread(fil)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    ret, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    network = False
    structureless = False
    branched_streaks = False
    dots = False
    globules = False

    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 20:
            continue
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = float(w) / h

        if aspect_ratio > 5:
            network = True
        elif aspect_ratio < 1.5:
            if area > 50:
                structureless = True
            elif area > 20:
                dots = True
        elif aspect_ratio < 2.5:
            if area > 70:
                branched_streaks = True
        elif aspect_ratio < 5:
            if area > 70:
                globules = True

    score = 0
    if network:
        score += 1
    if structureless:
        score += 1
    if branched_streaks:
        score += 1
    if dots:
        score += 1
    if globules:
        score += 1

    print(f"Dermoscopic score: {score}")
    d.append(score)


In [ ]:
import os
output_dir1 = '/content/drive/MyDrive/total1/benign'
output_dir2 = '/content/drive/MyDrive/total1/Melanoma'
if not os.path.exists(output_dir1):
 os.makedirs(output_dir1)
if not os.path.exists(output_dir2):
 os.makedirs(output_dir2)

input_dir = '/content/drive/MyDrive/output_directory3'

i=0;
for filename in os.listdir(input_dir):
   img = cv2.imread(os.path.join(input_dir, filename))
   ABCD[i]=float(a[i])*1.3+float(b[i])*0.1+float(c[i])*0.5+float(d[i])*0.5
   print(ABCD[i])
   if(ABCD[i]>4.45):
     cv2.imwrite(os.path.join(output_dir1, filename), img)
   else:
     cv2.imwrite(os.path.join(output_dir2, filename), img)
   i=i+1

In [ ]:
import os
import csv
# Set up the directory paths for the two types of images
class_1_dir = "/content/drive/MyDrive/total1/benign"
class_2_dir = "/content/drive/MyDrive/total1/Melanoma"
# Create a list to store the image file paths and labels
data = []
# Loop through the images in class 1 directory and add them to the data list with label 0
for img_file in os.listdir(class_1_dir):
    img_path = os.path.join(class_1_dir, img_file)
    data.append((img_path, 0))
# Loop through the images in class 2 directory and add them to the data list with label 1
for img_file in os.listdir(class_2_dir):
    img_path = os.path.join(class_2_dir, img_file)
    data.append((img_path, 1))
# Shuffle the data to randomize the order of the images
import random
random.shuffle(data)
# Write the data to a CSV file
with open("binary_classification_dataset.csv", "w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["image_path", "label"])
    for img_path, label in data:
        writer.writerow([img_path, label])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import cv2
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras import models
from keras.optimizers import Adam
from keras.optimizers import SGD
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
drive.mount('/content/gdrive')

# Load the image data and labels
images = np.load('/content/gdrive/MyDrive/images.npy')
labels = np.load('/content/gdrive/MyDrive/labels.npy')

# Set the image dimensions and batch size
img_height, img_width = 224, 224
batch_size = 16

# Split the data into training, validation, and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)
# Define the VGG16 model and add additional layers
vgg_model = Sequential()

pretrained_model = VGG16(include_top=False,
                   input_shape=(224, 224, 3),
                   pooling='avg',classes=2,
                   weights='imagenet')
# Unfreeze the layers in the pre-trained model
for layer in pretrained_model.layers[:-10]:
    layer.trainable = False
vgg_model.add(pretrained_model)
vgg_model.add(layers.Flatten())
vgg_model.add(layers.Dense(1024, activation='relu'))
vgg_model.add(layers.Dropout(0.5))
vgg_model.add(layers.Dense(1024, activation='relu'))
vgg_model.add(layers.Dropout(0.5))
vgg_model.add(layers.Dense(1, kernel_regularizer=l2(0.01), activation='linear'))
# Compile the model
opt = SGD(learning_rate=0.0001,momentum=0.9)
vgg_model.compile(optimizer=opt, loss='hinge', metrics=['accuracy'])

# Set the path to save the model in your Google Drive
model_path = '/content/gdrive/MyDrive/Models/vggfinetune.h5'
os.makedirs(os.path.dirname(model_path), exist_ok=True)
model_checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', save_best_only=True)
# Train the model with data augmentation
history=vgg_model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=100, callbacks=[model_checkpoint])

In [ ]:
X_train_features = vgg_model.predict(X_train)
X_test_features = vgg_model.predict(X_test)
print(X_test_features[:10])

In [ ]:
y_pred=[]
for i in X_test_features:
  if(i>0):
    y_pred.append(1);
  else:
    y_pred.append(0);
# Compute the accuracy of the SVM classifier
accuracy = accuracy_score(Y_test,y_pred)
print(accuracy)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(Y_test, y_pred, target_names=['mel','ben']))

print(confusion_matrix(Y_test, y_pred))

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Compute the false positive rate (FPR) and true positive rate (TPR) for different thresholds
fpr, tpr, thresholds = roc_curve(Y_test, y_pred)

# Compute the area under the ROC curve (AUC)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
drive.mount('/content/gdrive')
# Load the dataset
data_dir = '/content/gdrive/MyDrive/total1'
batch_size = 16
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Define the model
base_model = MobileNet(include_top=False, input_shape=(224, 224, 3), pooling='avg')
x = Dense(1, activation='sigmoid')(base_model.output)
model = Model(inputs=base_model.input, outputs=x)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    verbose=1,
    validation_steps=len(val_generator),
    steps_per_epoch=len(train_generator)
)

# Evaluate the model
test_loss, test_acc = model.evaluate(val_generator, verbose=1)
print('Test accuracy:', test_acc)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
import cv2
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split, KFold
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from keras import models
from keras.optimizers import Adam
from google.colab import drive
from tensorflow.keras.regularizers import l2
from keras import models
drive.mount('/content/drive')
images = np.load('/content/drive/MyDrive/images.npy')
labels = np.load('/content/drive/MyDrive/labels.npy')
img_height,img_width = 224,224
batch_size=40
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kfold.split(images, labels)):
    print(f"Fold: {fold}")
    X_train, X_val = images[train_idx], images[val_idx]
    Y_train, Y_val = labels[train_idx], labels[val_idx]

    resnet_model = Sequential()
    pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',classes=2,
                   weights='imagenet')
    resnet_model.add(pretrained_model)
    resnet_model.add(Flatten())
    resnet_model.add(Dense(512,activation='relu'))
    resnet_model.add(Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.01),activation='linear'))

    for layer in pretrained_model.layers[:-16]:
        layer.trainable = False

    opt = Adam(learning_rate=0.001)
    resnet_model.compile(optimizer = 'adam', loss = 'hinge', metrics = ['accuracy'])

    # Set the path to save the model in your Google Drive
    model_path = f'/content/drive/MyDrive/Models/new_fold{fold}.h5'
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    model_checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', save_best_only=True)

    history = resnet_model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_val, Y_val), epochs=100, callbacks=[model_checkpoint])


In [ ]:
# Plot the training and validation accuracy over epochs
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
y_pred=[]
for i in X_test_features:
  if(i>0):
    y_pred.append(1);
  else:
    y_pred.append(0);
# Compute the accuracy of the SVM classifier
accuracy = accuracy_score(Y_test,y_pred)
print(accuracy)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(Y_test, y_pred, target_names=['mel','ben']))

print(confusion_matrix(Y_test, y_pred))

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Compute the false positive rate (FPR) and true positive rate (TPR) for different thresholds
fpr, tpr, thresholds = roc_curve(Y_test, y_pred)

# Compute the area under the ROC curve (AUC)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()